# 환경설정

In [ ]:
# 패키지 영구 저장
import os, sys

# 구글 마운트 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd 
import re
import pickle
from collections import Counter
from wordcloud import WordCloud
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns


plt.rc('font', family='NanumGothic')            # matplotllib 한글 문장
mpl.rcParams['axes.unicode_minus'] = False      # minus 처리

file_path = '/content/drive/MyDrive/3조_XXXX/04. 기타자료/data/'

# 데이터 불러오기

In [ ]:
# 데이터 열기 
with open(file_path + 'kiwi_dataframe.pkl', 'rb') as f:
    talk_df = pickle.load(f)

talk_df[['문장','형태소','MORPHS']].loc[20:30,:]

,문장,형태소,MORPHS
20,의사들은 좋겠다 돈 많이 벌어서,"[(의사, NNG), (들, XSN), (좋다, VA), (겠, EP), (다, E...","[의사, 들, 좋다, 돈, 많이, 벌다]"
21,근데 병원에 있으면 너무 우울할듯,"[(근데, MAJ), (병원, NNG), (있다, VV), (으면, EC), (너무...","[근데, 병원, 있다, 너무, 우울, 하, 듯]"
22,1년 반 동안 한 거라곤 세금 퍼주기랑 거리두기밖에 없네,"[(1, SN), (년, NNB), (반, NNG), (동안, NNG), (하다, ...","[1, 년, 반, 동안, 하다, 거, 이다, 세금, 퍼주기, 거리두기, 없다]"
23,지금껏 포퓰리즘 정책밖에 없음,"[(지금껏, MAG), (포퓰리즘, NNG), (정책, NNG), (없다, VA),...","[지금껏, 포퓰리즘, 정책, 없다]"
24,난 그래도 좋은 사람이였구나,"[(나, NP), (그렇다, VA), (어도, EC), (좋다, VA), (은, E...","[나, 그렇다, 좋다, 사람, 이다]"
25,비록 여돌가지고 딸치고 놀앗지만 여자에 미쳐서 남자 패지는 않앗으니까 당당하게 살겠...,"[(비록, MAG), (여돌, NNP), (가지다, VV), (고, EC), (딸,...","[비록, 여돌, 가지다, 딸, 놀다, 여자, 미치다, 남자, 패다, 않다, 당당, ..."
26,쟤 이번에 트레이너 회원수 랭킹 1위 찍었다며?,"[(쟤, NP), (이번, NNG), (트레이너, NNP), (회원, NNG), (...","[쟤, 이번, 트레이너, 회원, 수, 랭킹, 1, 위, 찍다]"
27,기생오래비같이 뺀질반반해서 기지배들 취향엔 제격일 뿐 ㅉㅉ,"[(기생오래비, NNP), (같이, MAG), (빼다, VV), (ᆫ, ETM), ...","[기생오래비, 같이, 빼다, 질, 반반, 하, 기지배, 들, 취향, 제격, 이다, ..."
28,쟤는 음악으로 성공하기가 쉬운줄 아나?,"[(쟤, NP), (음악, NNG), (성공, NNG), (하, XSV), (기, ...","[쟤, 음악, 성공, 하, 쉽다, 줄, 알다]"
29,그니까 음악한다고 집에만 박혀서 부모님 등골빨아먹는게 참 재수없다,"[(그니까, MAJ), (음악, NNG), (하, XSV), (ᆫ다고, EC), (...","[그니까, 음악, 하, 집, 박히다, 부모님, 등골, 빨아먹다, 것, 참, 재수, 없다]"


# 집합으로 표현하기

In [ ]:
def ExtractText(series):
    text_lst = []
    for sent in series:
        text_lst.extend(sent)
    return text_lst 

text_none = ExtractText(talk_df.loc[talk_df['분류']=='비악플',:]['형태소'])
text_advisory = ExtractText(talk_df.loc[talk_df['분류']=='권고',:]['형태소'])
text_caution = ExtractText(talk_df.loc[talk_df['분류']=='주의',:]['형태소'])
text_warning = ExtractText(talk_df.loc[talk_df['분류']=='경고',:]['형태소'])

In [ ]:
set_none = set(text_none)
set_advisory = set(text_advisory)
set_caution = set(text_caution)
set_warning = set(text_warning)

In [ ]:
print(f'"비악플"의 형태소는 {len(set_none)} 개 입니다.')
print(f'"권고(비난/차별/혐오)"의 형태소는 {len(set_advisory)} 개 입니다.')
print(f'"주의(욕설/범죄)"의 형태소는 {len(set_caution)} 개 입니다.')
print(f'"경고(선정/폭력)"의 형태소는 {len(set_warning)} 개 입니다.')

"비악플"의 형태소는 25094 개 입니다.
"권고(비난/차별/혐오)"의 형태소는 27310 개 입니다.
"주의(욕설/범죄)"의 형태소는 10842 개 입니다.
"경고(선정/폭력)"의 형태소는 12985 개 입니다.


In [ ]:
set1 = set_none & set_advisory & set_caution & set_warning
set12 = set_advisory & set_caution & set_warning - set1
set2 = set_none - set_advisory - set_caution - set_warning
set3 = (set_none & set_caution) - set_advisory - set_warning
set4 = (set_none & set_advisory) - set_caution - set_warning 
set5 = (set_none & set_warning) - set_advisory - set_caution 
set6 = (set_none & set_caution) - set_warning - set3 
set7 = (set_none & set_advisory) - set1 - set4 - set6
set8 = (set_none & set_warning) - set1 - set5 - set7 
set9 = (set_advisory & set_caution) - set1 - set6 - set12
set10 = (set_advisory & set_warning) - set1 - set7 - set12 
set11 = (set_caution & set_warning) - set1 - set8 - set12
set13 = set_advisory - set_caution - set_warning - set_none
set14 = set_caution - set_warning - set_advisory - set_none
set15 = set_warning - set_advisory - set_caution - set_none

In [ ]:
print(f'비악플, 권고, 주의, 경고에서 모두 쓰인 형태소는 {len(set1)} 개 입니다.')
print(f'비악플에서만 쓰인 형태소는 {len(set2)} 개 입니다.')
print(f'권고에서만 쓰인 형태소는 {len(set13)} 개 입니다.')
print(f'주의에서만 쓰인 형태소는 {len(set14)} 개 입니다.')
print(f'경고에서만 쓰인 형태소는 {len(set15)} 개 입니다.')
print(f'비악플과 주의에서만 쓰인 형태소는 {len(set3)} 개 입니다.')
print(f'비악플과 권고에서만 쓰인 형태소는 {len(set4)} 개 입니다.')
print(f'비악플과 경고에서만 쓰인 형태소는 {len(set5)} 개 입니다.')
print(f'비악플, 권고, 주의에서만 쓰인 형태소는 {len(set6)} 개 입니다.')
print(f'비악플, 권고, 경고에서만 쓰인 형태소는 {len(set7)} 개 입니다.')
print(f'비악플, 주의, 경고에서만 쓰인 형태소는 {len(set8)} 개 입니다.')
print(f'권고, 주의에서만 쓰인 형태소는 {len(set9)} 개 입니다.')
print(f'권고, 경고에서만 쓰인 형태소는 {len(set10)} 개 입니다.')
print(f'주의, 경고에서만 쓰인 형태소는 {len(set11)} 개 입니다.')
print(f'권고, 주의, 경고(악플 모두)에서 쓰인 형태소는 {len(set12)} 개 입니다.')

비악플, 권고, 주의, 경고에서 모두 쓰인 형태소는 5638 개 입니다.
비악플에서만 쓰인 형태소는 7106 개 입니다.
권고에서만 쓰인 형태소는 8387 개 입니다.
주의에서만 쓰인 형태소는 1584 개 입니다.
경고에서만 쓰인 형태소는 2037 개 입니다.
비악플과 주의에서만 쓰인 형태소는 345 개 입니다.
비악플과 권고에서만 쓰인 형태소는 6319 개 입니다.
비악플과 경고에서만 쓰인 형태소는 700 개 입니다.
비악플, 권고, 주의에서만 쓰인 형태소는 1893 개 입니다.
비악플, 권고, 경고에서만 쓰인 형태소는 2988 개 입니다.
비악플, 주의, 경고에서만 쓰인 형태소는 105 개 입니다.
권고, 주의에서만 쓰인 형태소는 724 개 입니다.
권고, 경고에서만 쓰인 형태소는 964 개 입니다.
주의, 경고에서만 쓰인 형태소는 156 개 입니다.
권고, 주의, 경고(악플 모두)에서 쓰인 형태소는 397 개 입니다.


# 단독을 키워드로 유사도 분석하기(set13, set14, set15)

## Fast Text 함수

In [ ]:
from gensim.models import FastText
from gensim.models import Word2Vec

model = FastText(sentences=talk_df['MORPHS'], window=5, min_count=5, workers=4, sg=1)

In [ ]:
print(len(list(model.wv.vocab)))

12978


## 기존에 있는 거 수정하기기

In [ ]:
dic_df = pd.read_excel(file_path + '혐오사전_유림.xlsx')
dic_df

,keyword,빈도기반 tag,비악플,선정,폭력,혐오
0,짱꼴라,혐오,0,0,2,27
1,정자은행,혐오,2,0,0,3
2,띨빡,혐오,0,0,3,6
3,뇌가리,혐오,0,0,2,6
4,메퇴지,혐오,0,0,1,5
...,...,...,...,...,...,...
2284,ㅅㅌㅊ,선정,6,6,0,0
2285,와씨,선정,4,4,0,0
2286,스와핑,선정,3,2,0,0
2287,볼륨감,선정,3,3,0,0


In [ ]:
key

'뻐기다'

In [ ]:
talk_df.loc[talk_df['MORPHS'].apply(lambda x: key in x),['분류']].value_counts()

{}

In [ ]:
# 새로운 태그 만들기
cat_lst = ['비악플','권고','주의','경고']
    
cnt_lst = []
print('Start!', end=' ')
i = 0
for key in dic_df['keyword'].tolist():
    # print(key)
    counts = talk_df.loc[talk_df['MORPHS'].apply(lambda x: key in x),['분류']].value_counts()
    counts = dict([(cat[0], cnt) for cat, cnt in counts.items()])
    if counts == {}:
      max_cat = 0
    else:
      max_cat = list(counts.keys())[0]
    # print(counts)
    if len(counts) != len(cat_lst):
        no_cat = [cat for cat in cat_lst if cat not in counts.keys()]
        for cat in no_cat:
            counts[cat] = 0
    
    cnt_lst.append([max_cat] + [cnt for cat, cnt in sorted(counts.items())])

    # 메세지 출력 
    i += 1
    if i % 200 == 0:
        print(f'> {i}', end=' ')
    if i % 2000 == 0:
        print()
print('End!')

cnt_df = pd.DataFrame(cnt_lst, columns=['빈도기반 tag'] + cat_lst)
dic_df = pd.concat([dic_df, cnt_df], axis=1)
dic_df

Start! > 200 > 400 > 600 > 800 > 1000 > 1200 > 1400 > 1600 > 1800 > 2000 
> 2200 End!


,keyword,빈도기반 tag,비악플,선정,폭력,혐오,빈도기반 tag,비악플,권고,주의,경고
0,짱꼴라,혐오,0,0,2,27,권고,0,30,0,2
1,정자은행,혐오,2,0,0,3,권고,0,3,2,0
2,띨빡,혐오,0,0,3,6,권고,0,5,0,1
3,뇌가리,혐오,0,0,2,6,권고,2,6,0,0
4,메퇴지,혐오,0,0,1,5,권고,0,5,0,1
...,...,...,...,...,...,...,...,...,...,...,...
2284,ㅅㅌㅊ,선정,6,6,0,0,경고,6,0,6,0
2285,와씨,선정,4,4,0,0,비악플,2,0,4,0
2286,스와핑,선정,3,2,0,0,비악플,2,0,3,0
2287,볼륨감,선정,3,3,0,0,경고,3,0,3,0


In [ ]:
dic_df.to_excel(file_path + 'yurim_dic.xlsx')

## 단어 불러ㅗ기기

In [ ]:
set13_nouns = [pos[0] for pos in set13 if pos[1][0]=='N']
set14_nouns = [pos[0] for pos in set14 if pos[1][0]=='N']
set15_nouns = [pos[0] for pos in set15 if pos[1][0]=='N']
print(set13_nouns)
print(set14_nouns)
print(set15_nouns)

['상동', '프레디', '카테고리', '멱따', '미드필더', '마산어시장', '똥트남', '사익', '보력', '마스터', '똥푸산', '공정성', '승려', '껄떡거리네', '동점자', '남동', '개패야됨', '박하사탕', '지방러들', '살모사', '바이크충', '오또캐충', '디룩디룩쪄서', '메투기꺼져', '파냐', '환빠들', '개부럽노', '국회의사당', '차노', '모근', '성전', '막차', '위험군', '샤리아', '개축', '개파', '쥐며느리', '물장사', '하더', '결부', '키스트', '프로참견러', '가관이네', '고동', '제', '구석기', '쎄야', '도라방스', '성자', '개무시하네', '핑돼', '동장', '거잖음', '지잡대생', '막사', '꿰서', '지이이이잉', '개많네', '박사모', '인주', '지성체', '저래서', '추접', '싸이버렉카충들', '간질', '취두부', '냉각기', '필테', '지하철도', '여생', '난대', '카스', '반일충', '워리어짓', '찌릉', '우상호', '포석', '문준', '페티쉬', '씽크대', '사리사욕', '웜퇘지', '쩔구나', '꼬랑내', '저촉', '의약', '윤어벙', '순례', '마태', '햄최', '애늙으니', '성주', '그릉뱅이', '저장', '사짜', '공산당원', '어민', '냄창임', '글른', '떵거충', '자멸', '모성', '꼬다', '아동폭력', '지구력', '돌풍', '궐기', '언론사', '왕발', '모로쇠', '티내지마', '적임자', '나불대더', '미국판', '타박', '남초사이트', '페미슬람', '슈가보이', '우선순위', '빡추', '차박러', '노예지', '니혼진', '쌈질', '밤탱', '목동', '피의', '클래스', '현란', '소래포구', '손투리', '빈국', '배째', '재기해라잉', '밥버거', '찌뿌린', '바이든', '입관', '도적', '산물', '넙데데함', '숭례문', '뇌도', '특임', '심해어

## 키워드 추출하기 

In [ ]:
# 방법 : 키워드 리스트를 넣었을 때 상위 n개 가져오기
# input: keywordlist=[(keyword1, tag1), (keyword2, tag2), ...]
def ExtractSimilar(word_lst, n, model_name):
    sim_lst = []
    sim_set = set()
    for key, tag in word_lst:
        sim = model_name.wv.most_similar(key, topn=n)
        for word, p in sim:
            if word[0].isdigit()==False and len(word) > 1 and word not in sim_set:
               sim_lst.append([word, key, round(p,4), tag, len(word)])
               sim_set.add(word)

    sim_df = pd.DataFrame(sim_lst, columns=['keyword', 'search', 'similarity', 'tag', 'len'])
    sim_df.sort_values(by=['search','keyword','len','similarity'], inplace=True)
    sim_df.reset_index(drop=True, inplace=True)
    
    return sim_df


### 1) 권고(Advisory, 비난/혐오/차별)

In [ ]:
# 반복 함수
advisory_lst=[(noun, '권고') for noun in set13_nouns if len(noun) > 1 and noun in model.wv.vocab.keys()]
rep = 5
print('Start!', end=' ')
for i in range(rep):
    dic_df = ExtractSimilar(advisory_lst, n=10, model_name=model)
    print(f'> {dic_df.shape[0]}', end=' ')

    advisory_lst=[(noun, '권고') for noun in dic_df['keyword'] if len(noun) > 1 and noun in model.wv.vocab.keys()]
print('End!')
dic_df

Start! > 2696 > 5268 > 7005 > 8107 > 8741 End!


,keyword,search,similarity,tag,len
0,av,AV,0.8331,권고,2
1,상호,AV,0.8117,권고,2
2,포르노,AV,0.8056,권고,3
3,반도체,IT,0.9144,권고,3
4,삼성전자,IT,0.9088,권고,4
...,...,...,...,...,...
8736,장어,힘겹다,0.9092,권고,2
8737,졸부,힙찔이,0.7924,권고,2
8738,패턴,힙찔이,0.8106,권고,2
8739,힙합,힙찔이,0.7983,권고,2


In [ ]:
dic_df = pd.read_csv(file_path + 'yulim_dic.csv')

FileNotFoundError: ignored

In [ ]:
# 새로운 태그 만들기
cat_lst = ['비악플','권고','주의','경고']
    
cnt_lst = []
print('Start!', end=' ')
i = 0
for key in dic_df['keyword'].tolist():
    counts = talk_df.loc[talk_df['MORPHS'].apply(lambda x: key in x),['분류']].value_counts()
    counts = dict([(cat[0], cnt) for cat, cnt in counts.items()])
    max_cat = list(counts.keys())[0]
    # print(counts)
    if len(counts) != len(cat_lst):
        no_cat = [cat for cat in cat_lst if cat not in counts.keys()]
        for cat in no_cat:
            counts[cat] = 0
    
    cnt_lst.append([max_cat] + [cnt for cat, cnt in sorted(counts.items())])

    # 메세지 출력 
    i += 1
    if i % 200 == 0:
        print(f'> {i}', end=' ')
    if i % 2000 == 0:
        print()
print('End!')

cnt_df = pd.DataFrame(cnt_lst, columns=['빈도기반 tag'] + cat_lst)
dic_df = pd.concat([dic_df, cnt_df], axis=1)
dic_df

### 2) 주의(Caution, 욕설/범죄)

In [ ]:
# 반복 함수
caution_lst=[(noun, '주의') for noun in set14_nouns if len(noun) > 1 and noun in model.wv.vocab.keys()]
rep = 5
print('Start!', end=' ')
for i in range(rep):
    dic_df = ExtractSimilar(caution_lst, n=10, model_name=model)
    print(f'> {dic_df.shape[0]}', end=' ')

    caution_lst=[(noun, '주의') for noun in dic_df['keyword'] if len(noun) > 1 and noun in model.wv.vocab.keys()]
print('End!')
dic_df

In [ ]:
# 새로운 태그 만들기
cat_lst = ['비악플','권고','주의','경고']
    
cnt_lst = []
print('Start!', end=' ')
i = 0
for key in dic_df['keyword'].tolist():
    counts = talk_df.loc[talk_df['MORPHS'].apply(lambda x: key in x),['분류']].value_counts()
    counts = dict([(cat[0], cnt) for cat, cnt in counts.items()])
    max_cat = list(counts.keys())[0]
    # print(counts)
    if len(counts) != len(cat_lst):
        no_cat = [cat for cat in cat_lst if cat not in counts.keys()]
        for cat in no_cat:
            counts[cat] = 0
    
    cnt_lst.append([max_cat] + [cnt for cat, cnt in sorted(counts.items())])

    # 메세지 출력 
    i += 1
    if i % 200 == 0:
        print(f'> {i}', end=' ')
    if i % 2000 == 0:
        print()
print('End!')

cnt_df = pd.DataFrame(cnt_lst, columns=['빈도기반 tag'] + cat_lst)
dic_df = pd.concat([dic_df, cnt_df], axis=1)
dic_df

### 3) 경고(Warning, 선정/폭력)

In [ ]:
# 반복 함수
warning_lst=[(noun, '경고') for noun in set15_nouns if len(noun) > 1 and noun in model.wv.vocab.keys()]
rep = 5
print('Start!', end=' ')
for i in range(rep):
    dic_df = ExtractSimilar(warning_lst, n=10, model_name=model)
    print(f'> {dic_df.shape[0]}', end=' ')

    warning_lst=[(noun, 경고') for noun in dic_df['keyword'] if len(noun) > 1 and noun in model.wv.vocab.keys()]
print('End!')
dic_df

In [ ]:
# 새로운 태그 만들기
cat_lst = ['비악플','권고','주의','경고']
    
cnt_lst = []
print('Start!', end=' ')
i = 0
for key in dic_df['keyword'].tolist():
    counts = talk_df.loc[talk_df['MORPHS'].apply(lambda x: key in x),['분류']].value_counts()
    counts = dict([(cat[0], cnt) for cat, cnt in counts.items()])
    max_cat = list(counts.keys())[0]
    # print(counts)
    if len(counts) != len(cat_lst):
        no_cat = [cat for cat in cat_lst if cat not in counts.keys()]
        for cat in no_cat:
            counts[cat] = 0
    
    cnt_lst.append([max_cat] + [cnt for cat, cnt in sorted(counts.items())])

    # 메세지 출력 
    i += 1
    if i % 200 == 0:
        print(f'> {i}', end=' ')
    if i % 2000 == 0:
        print()
print('End!')

cnt_df = pd.DataFrame(cnt_lst, columns=['빈도기반 tag'] + cat_lst)
dic_df = pd.concat([dic_df, cnt_df], axis=1)
dic_df